# Finetune RuGPTs in huggingface
How to finetune RuGPTs models with huggingface. Example for RuGPT3Small. Nfor other models it will take more GPU memory.

This notebook is valid for all RuGPTs models except RuGPT3XL.
## Install env

In [ ]:
!pip3 install transformers==3.5.0

In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

In [4]:
!mkdir models/

## Download files

In [ ]:
!wget -O train.txt https://www.dropbox.com/s/oa3v9c7g9bp40xw/train.txt?dl=0
!wget -O valid.txt https://www.dropbox.com/s/mworl3ld6r3bg62/valid.txt?dl=0

## Train 
The following code download our model and tokenizer from huggingface and finetune model for generating essays.

This took aroung ten minutes and obtain perplexity = 13-16

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
    --output_dir=models/essays \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --block_size 2048 \
    --overwrite_output_dir

## Evaluate model
### Generate with generate_transformers.py script.

In [8]:
!python ru-gpts/generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=models/essays \
    --k=5 \
    --p=0.95 \
    --length=500 \
    --repetition_penalty=5

2021-02-11 12:14:52.166691: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
02/11/2021 12:15:04 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=5, length=500, model_name_or_path='models/essays', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.95, padding_text='', prompt='', repetition_penalty=5.0, seed=42, stop_token='</s>', temperature=1.0, xlm_language='')
Context >>> <s>Тема: «Создает человека природа, но развивает и образует его общество». (В.Т. Белинский)\nСочинение: 
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
ruGPT:
<s>Тема: «Создает человека природа, но развивает и образует его общество». (В.Т. Белинский)\nСочинение:  В данном высказывании автор поднимает проблему личности как носителя культуры.</S
Личность - это совокупность всех индивидуально значимых свойств индивидуальности («мораль», духовная культура). Личность выступает в качестве субъект

### Generate mannualy

In [30]:
import numpy as np
import torch

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [9]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [10]:
tok = GPT2Tokenizer.from_pretrained("models/essays")

In [11]:
model = GPT2LMHeadModel.from_pretrained("models/essays")

In [ ]:
model.cuda()

In [32]:
text = "<s>Тема: «Создает человека природа, но развивает и образует его общество». (В.Т. Белинский)\nСочинение: "
inpt = tok.encode(text, return_tensors="pt")

In [ ]:
out = model.generate(inpt.cuda(), max_length=500, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=1)

In [36]:
tok.decode(out[0])

'<s>Тема: «Создает человека природа, но развивает и образует его общество». (В.Т. Белинский)\nСочинение: _Эссе №1«Цель общества есть удовлетворение потребностей» — девиз философов всех времен ной поры («Мы должны стремиться к счастью»). И действительно цели разные бывают у обществ – это потребности удовлетворения естественных влечений или же социально-бытовые нужды; удовлетворены ли эти желания достигнуты благодаря деятельности людей - другой вопрос… В обществе господствует биологическая потребность в наслаждении жизнью(Ж.- Жорес), а социальная связана с потреблением материальных благ для существования.</SOCTERN’Я полностью согласна со высказыванием Акакия Львовича Толстого о том,что цель любого государства состоит не только во власти народа над государством. Но главной целью является само государство! Именно оно определяет жизнь страны как стабильную систему правовых отношений между людьми без принуждения властей.(АП РФ). Я считаю что Толстой прав по этому поводу.. Действительно челов